In [4]:
import pathlib
import numpy as np

from aiida import orm, load_profile
from aiida.engine import submit

from qe_tools import CONSTANTS

from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

# BaTiO3
Let's define the BTO structure to use as example for this tutorial.

In [5]:
cell = [
    [5.34198, -3.08419, 4.38073],
    [0.00000,  6.16839, 4.38073],
    [-5.3419, -3.08419, 4.38073],
]
cell = CONSTANTS.bohr_to_ang*np.array(cell)

structure = orm.StructureData(cell=cell)

structure.append_atom(position=np.dot([0.48674, 0.4867, 0.4867], cell),  symbols='Ti')
structure.append_atom(position=np.dot([0.51163, 0.5116, 0.0192], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.51163, 0.0192, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.01929, 0.5116, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.99899, 0.9989, 0.9989], cell),  symbols='Ba')

## HubbardStructureData initialization
Let's initialize the HubbardStructureData with the BTO structure!

> Important: You only need to do these steps once! Later just keep track of this testing structure via the PK and reload it from the database so you don't get 100 BaTiO3 strucures in your storage. ^^

In [6]:
hubbard_data = HubbardStructureData.from_structure(structure=structure)

In [7]:
hubbard_data

<HubbardStructureData: uuid: e1aa6e73-128f-4f53-b384-ec57edc902f2 (unstored)>

## Initializing the on-site Hubbard
Let's initialize the on-site Hubbard parameter for the titanium atom.

In [8]:
hubbard_data.initialize_onsites_hubbard('Ti', '3d', 5.0) # index 0 for Ti ==> Hubbard U

Here how it is stored in the class.

In [13]:
hubbard_data

<HubbardStructureData: uuid: e1aa6e73-128f-4f53-b384-ec57edc902f2 (unstored)>

## Initializing the inter-site Hubbard
Let's initialize the inter-site Hubbard parameter for the titanium and oxygen atoms.

In [16]:
hubbard_data.initialize_intersites_hubbard(
    atom_name='Ti', atom_manifold='3d', neighbour_name='O', neighbour_manifold='2p', value=0.0001
    )

        # self,
        # atom_name: str,
        # atom_manifold: str,
        # neighbour_name: str,
        # neighbour_manifold: str,
        # value: float = 1e-8,
        # hubbard_type: str = "V",
        # use_kinds: bool = True,


The parameters are saved in the property `hubbard_parameters` as a list.

In [18]:
hubbard_data.hubbard

Hubbard(parameters=[HubbardParameters(atom_index=0, atom_manifold='3d', neighbour_index=0, neighbour_manifold='3d', translation=(0, 0, 0), value=5.0, hubbard_type='Ueff'), HubbardParameters(atom_index=0, atom_manifold='3d', neighbour_index=1, neighbour_manifold='2p', translation=(0, 0, 0), value=0.0001, hubbard_type='V')], projectors='ortho-atomic', formulation='dudarev')

In [20]:
hubbard_data.store()

<HubbardStructureData: uuid: e1aa6e73-128f-4f53-b384-ec57edc902f2 (pk: 19376)>

In [21]:
hubbard_data.pk

19376

## Running the `SelfConsistentHubbardWorkChain`

In [22]:
hubbard_data = orm.load_node(19376)  # I load the node from the database instead of always regenerating it
# print(hubbard_data.get_quantum_espresso_hubbard_card())

In [23]:
pw_code = orm.load_code(2182)
hp_code = orm.load_code(2183)

builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='fast',
    # overrides=pathlib.Path('hubbard_overrides.yaml')
)

builder.skip_first_relax = True

In [24]:
submit_node = submit(builder)
print(submit_node)

uuid: 5cbcdb17-012e-48a5-b525-37a2a77430d3 (pk: 19398) (aiida.workflows:quantumespresso.hp.hubbard)
